In [1]:
import requests
from bs4 import BeautifulSoup

def get_tweet(url):
    headers = {
        'User-Agent': 'Twitterbot'
    }	
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        tweet_text = soup.find('meta', attrs={'property': 'og:description'}).get('content')
        if tweet_text:
            return tweet_text
        else:
            raise Exception('Error: Could not find tweet text')
    else:
        raise Exception('Error: Status code', response.status_code)

In [29]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

sarcasm_model = tf.keras.models.load_model('models/sarcasm/v2/v2.h5')
positivity_model = tf.keras.models.load_model('models/positivity/v2/v2.h5')

sarcasm_tokenizer_path = 'models/sarcasm/tokenizer.pickle'
positivity_tokenizer_path = 'models/positivity/tokenizer.pickle'

with open(sarcasm_tokenizer_path, 'rb') as handle:
    sarcasm_tokenizer = pickle.load(handle)

with open(positivity_tokenizer_path, 'rb') as handle:
    positivity_tokenizer = pickle.load(handle)
    
def predict_sarcasm(tweet):
    tweet = [tweet]
    tweet = sarcasm_tokenizer.texts_to_sequences(tweet)
    tweet = pad_sequences(tweet, maxlen=120, padding='post')
    prediction = sarcasm_model.predict(tweet)
    return prediction
    
def predict_positivity(tweet):
    tweet = [tweet]
    tweet = positivity_tokenizer.texts_to_sequences(tweet)
    tweet = pad_sequences(tweet, maxlen=120, padding='post')
    prediction = positivity_model.predict(tweet)
    return prediction

    

In [36]:
TWEET_URL = 'https://twitter.com/elonmusk/status/1778434736454553625'

tweet = get_tweet(TWEET_URL)

sarcasm_prediction = predict_sarcasm(tweet)
positivity_prediction = predict_positivity(tweet)

print('Tweet:', tweet)
print('Sarcasm prediction:', sarcasm_prediction[0])
print('Positivity prediction:', positivity_prediction[0])

if sarcasm_prediction[0] > 0.5:
    print('Sarcasm detected!')
else:
    print('No sarcasm detected!')
    
if positivity_prediction[0] > 0.5:
    print('Positive tweet!')
else:
    print('Negative tweet!')
    
    

1/1 [==============================] - 0s 15ms/step
Tweet: The severity of the censorship and the degree to which Brazil’s own laws are being broken, to the detriment of their own people, is the worst of any country in the world in which this platform operates.
Sarcasm prediction: [4.7681488e-06]
Positivity prediction: [0.19872558]
No sarcasm detected!
Negative tweet!
